In [ ]:
# --- Importing Libraries
import sys
print("Python Executable:", sys.executable)

import pandas as pd # type: ignore
import numpy as np
np.int = int # 😠 come on pygam
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore
import statsmodels.api as sm # type: ignore
import statsmodels.formula.api as smf # type: ignore
import matplotlib.pyplot as plt # type: ignore
from stargazer.stargazer import Stargazer # type: ignore
from IPython.display import HTML, display # type: ignore
from sklearn.preprocessing import MinMaxScaler # type: ignore
import polars as pl # type: ignore
import re
import matplotlib.ticker as ticker
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, root_mean_squared_error
from pygam import LinearGAM, s
#Hacking together version deprecationsin pygam
import scipy.sparse
scipy.sparse.csr.csr_matrix.A = property(lambda self: self.toarray())
from scipy.stats import zscore



In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

dates = pd.date_range(start="2022-01-01", periods=1000, freq="D")

# Sales Data
store_sales = np.random.normal(loc=4500, scale=500, size=1000).clip(3000, 6000)
web_sales = np.random.normal(loc=3500, scale=600, size=1000).clip(2000, 5000)

# Traffic Data
store_traffic = np.random.normal(loc=5000, scale=1000, size=1000).clip(2000, 8000)
web_traffic = np.random.normal(loc=2_000_000, scale=1_000_000, size=1000).clip(500_000, 5_000_000)

# Create a dataframe
df = pd.DataFrame({
    'date': dates,
    'store_sales': store_sales.astype(int),
    'web_sales': web_sales.astype(int),
    'store_traffic': store_traffic.astype(int),
    'web_traffic': web_traffic.astype(int)
})

print(df.describe())

In [ ]:
sns.set(style="whitegrid")

# Variables to plot (excluding Date)
variables = ['store_sales', 'web_sales', 'store_traffic', 'web_traffic']

# Set up subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 8))
axes = axes.flatten()

for i, var in enumerate(variables):
    sns.histplot(data=df, x=var, kde=True, ax=axes[i], bins=30, color='steelblue')
    axes[i].set_title(f"Distribution of {var.replace('_', ' ').title()}")
    axes[i].set_xlabel("")
    axes[i].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
scaler = StandardScaler()

# Select the columns to scale (excluding 'Date')
outcome_vars = ['store_sales', 'web_sales', 'store_traffic', 'web_traffic']

# Fit and transform the data
df_scaled = df.copy()
df_scaled[outcome_vars] = scaler.fit_transform(df[outcome_vars])

In [ ]:
#Scaled Plots
sns.set(style="whitegrid")

# Variables to plot (excluding Date)
variables = ['store_sales', 'web_sales', 'store_traffic', 'web_traffic']

# Set up subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 8))
axes = axes.flatten()

for i, var in enumerate(variables):
    sns.histplot(data=df_scaled, x=var, kde=True, ax=axes[i], bins=30, color='steelblue')
    axes[i].set_title(f"Distribution of {var.replace('_', ' ').title()}")
    axes[i].set_xlabel("")
    axes[i].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
# --- Creating Outcomes

df['raw_combined_traffic'] = df['web_traffic'] + df['store_traffic']
df_scaled['sum_of_scaled_components'] = df_scaled['web_traffic'] + df_scaled['store_traffic']

scaler_traffic = StandardScaler()
df_scaled['scaled_combined_traffic'] = scaler_traffic.fit_transform(df[['raw_combined_traffic']])


In [ ]:
df['web_conversion'] = df['web_sales'] / df['web_traffic']

df['store_conversion'] = df['store_sales'] / df['store_traffic']

In [ ]:
# --- Creating Predictors

np.random.seed(123)

# Define the ranges you want
predictor_specs = [
    (2000, 5000),        # x1
    (25000, 50000),      # x2
    (50000, 100000),     # x3
    (500000, 1000000),   # x4
]

# Generate predictors with noise
for i, (low, high) in enumerate(predictor_specs):
    df[f'x{i+1}'] = np.random.uniform(low, high, size=len(df))

In [ ]:
variables = ['x1', 'x2', 'x3', 'x4']

# stdev of Y
traffic_std = df['raw_combined_traffic'].std()

# stdev of X's - dictionary for eassy access later
predictor_std = df[variables].std().to_dict()

# Scaling X's
input_scaler = StandardScaler()
df_scaled[variables] = input_scaler.fit_transform(df[variables])

In [ ]:
# --- Regressions
model_a = smf.ols("raw_combined_traffic ~ x1 + x2 + x3 + x4", data = df).fit(cov_type='HC1')

# Regressing Traffic scaled after adding
model_b= smf.ols("scaled_combined_traffic ~ x1 + x2 + x3 + x4", data = df_scaled).fit(cov_type='HC1')

In [ ]:
# --- 
#Regression coefficients
coeff_a_std = model_a.params[variables]
coeff_b_std = model_b.params[variables]

# Convert coefficients back - B*stdY/stdX
level_effects_b = {
    var: coeff_b_std[var] * (traffic_std / predictor_std[var])
    for var in variables
}


converted_b = pd.DataFrame({
    'Variable': variables,
    'Std_Coeff': [coeff_b_std[var] for var in variables],
    'Level_Effect': [level_effects_b[var] for var in variables],
    'Unscaled_Regression': [coeff_a_std[var] for var in variables]
})

print(converted_b)

In [ ]:
model_c = smf.ols("sum_of_scaled_components ~ x1 + x2 + x3 + x4", data = df_scaled).fit(cov_type='HC1')


In [ ]:
web_z = zscore(df['web_traffic'])
store_z = zscore(df['store_traffic'])

corr = np.corrcoef(web_z, store_z)[0, 1]

sd_y3 = np.sqrt(2 + 2 * corr)

print(f"Standard deviation of Y_3 (sum of standardized components): {sd_y3}")

In [ ]:
model_a.summary()